In [1]:
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev


In [2]:
#!pip install transformers

In [3]:
import torch_xla
import torch_xla.core.xla_model as xm

In [4]:
import pandas as pd
import numpy as np
import os
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# set a seed value
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

import transformers
from transformers import BertTokenizer, BertForSequenceClassification 
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import AdamW

import warnings
warnings.filterwarnings("ignore")


print(torch.__version__)

1.6.0a0+bf2bbd9


In [5]:
from transformers import BertTokenizer

# Instantiate the Bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [6]:
len(tokenizer.vocab)

30522

In [7]:
# The vocab is an ordered dictionary - key/value pairs.
# This is how to see which tokens are associated with a particular word.

bert_vocab = tokenizer.vocab

print(bert_vocab['[CLS]'])
print(bert_vocab['[SEP]'])
print(bert_vocab['[PAD]'])

print(bert_vocab['hello'])
print(bert_vocab['world'])

101
102
0
7592
2088


In [8]:
MAX_LEN = 10 # This value could be set as 256, 512 etc.

sentence1 = 'Hello there.'

encoded_dict = tokenizer.encode_plus(
            sentence1,                      # Sentence to encode.
            add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
            max_length = MAX_LEN,           # Pad or truncate.
            pad_to_max_length = True,
            truncation=True,
            return_attention_mask = True,   # Construct attn. masks.
            return_tensors = 'pt',          # Return pytorch tensors.
           )


encoded_dict

{'input_ids': tensor([[ 101, 7592, 2045, 1012,  102,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])}

In [9]:
#!pip install sentencepiece

In [10]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

MODEL_TYPE = 'xlm-roberta-base'

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)

In [11]:
tokenizer.vocab_size

250002

In [12]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'cls_token': '<s>',
 'eos_token': '</s>',
 'mask_token': '<mask>',
 'pad_token': '<pad>',
 'sep_token': '</s>',
 'unk_token': '<unk>'}

In [13]:
print('bos_token_id <s>:', tokenizer.bos_token_id)
print('eos_token_id </s>:', tokenizer.eos_token_id)
print('sep_token_id </s>:', tokenizer.sep_token_id)
print('pad_token_id <pad>:', tokenizer.pad_token_id)

bos_token_id <s>: 0
eos_token_id </s>: 2
sep_token_id </s>: 2
pad_token_id <pad>: 1


In [14]:
## Mount Drive into Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
dataset_path = "/content/drive/My Drive/Colab Notebooks/projeto-reconhecimento-autoria/dataset/"

In [16]:
data = pd.read_csv(dataset_path + "books_authorship_english.csv") 

In [17]:
data.shape

(78, 7)

In [18]:
min_size = 300

In [19]:
label_to_ix = {}
for label in data.label:
    for word in label.split():
        if word not in label_to_ix:
            label_to_ix[word]=len(label_to_ix)
label_to_ix

{'alan_poe': 3,
 'arthur_doyle': 9,
 'bram_stoker': 4,
 'charles_darwin': 8,
 'charles_dickens': 6,
 'daniel_defoe': 2,
 'george_eliot': 10,
 'hector_hugh': 0,
 'jane_austen': 11,
 'joseph_conrad': 12,
 'mark_twain': 5,
 'pelham_grenville': 7,
 'thomas_hardy': 1}

In [22]:
random_flag = False

In [23]:
df_train = pd.DataFrame(columns = ['text', 'label'])
corpus = [i.split() for i in data["words"]]
segmented_corpus = []

for book in corpus:
  partitions = int(round(len(book)/min_size,2) + 0.5)
  segments = [book[int(round(min_size * i)): int(round(min_size * (i + 1)))] for i in range(partitions)]
  segmented_corpus.append(segments)

for label, partitions in zip(data["label"], segmented_corpus):
  if random_flag:
    random_index = random.randint(0, len(partitions) - 1)
    text = " ".join(partitions[random_index])
    intent = label
    df_train = df_train.append({'text': text, 'label': intent}, ignore_index=True)
  else:
    for p in partitions:
      text = " ".join(p)
      intent = label
      df_train = df_train.append({'text': text, 'label': intent}, ignore_index=True)
df_train.tail()

,text,label
33343,we leave directly pay no attention to her cry ...,joseph_conrad
33344,come here look at abdulla now he life here bec...,joseph_conrad
33345,waste of money that devil only know what becom...,joseph_conrad
33346,do everybody finally she i verily believe come...,joseph_conrad
33347,speak very loud and thump the table i want to ...,joseph_conrad


In [24]:
df_train.shape

(33348, 2)

In [25]:
from sklearn.model_selection import KFold, StratifiedKFold

# shuffle
df = shuffle(df_train)

# initialize kfold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1024)

# for stratification
y = df['label']

# Note:
# Each fold is a tuple ([train_index_values], [val_index_values])
# fold_0, fold_1, fold_2, fold_3, fold_5 = kf.split(df, y)

# Put the folds into a list. This is a list of tuples.
fold_list = list(kf.split(df, y))

train_df_list = []
val_df_list = []

for i, fold in enumerate(fold_list):

    # map the train and val index values to dataframe rows
    df_train = df[df.index.isin(fold[0])]
    df_val = df[df.index.isin(fold[1])]
    
    train_df_list.append(df_train)
    val_df_list.append(df_val)
    
    

print(len(train_df_list))
print(len(val_df_list))


5
5


In [26]:
MODEL_TYPE = 'bert-base-multilingual-uncased'

NUM_FOLDS = 5

# Saving 5 TPU models will exceed the 4.9GB disk space.
# Therefore, will will only train on 3 folds.
NUM_FOLDS_TO_TRAIN = 3 

L_RATE = 1e-5
MAX_LEN = 256
NUM_EPOCHS = 3
BATCH_SIZE = 32
NUM_CORES = os.cpu_count()

NUM_CORES

2

In [27]:
# For TPU

device = xm.xla_device()

print(device)

xla:1


In [28]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE, do_lower_case=True)


Loading BERT tokenizer...


In [29]:
class BookDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'text']
        
        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    sentence1,           # Sentences to encode.
                    add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
                    max_length = MAX_LEN,           # Pad or truncate all sentences.
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )  
        
        # These are torch tensors already.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        token_type_ids = encoded_dict['token_type_ids'][0]
        
        # Convert the target to a torch tensor
        target = torch.tensor(label_to_ix[self.df_data.loc[index, 'label']])

        sample = (padded_token_list, att_mask, token_type_ids, target)


        return sample


    def __len__(self):
        return len(self.df_data)
    
    

In [30]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [31]:
train_data = BookDataset(df_train)
val_data = BookDataset(df_val)



train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

val_dataloader = torch.utils.data.DataLoader(val_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

In [32]:
print(len(train_dataloader))
print(len(val_dataloader))

834
209


In [33]:
# Get one train batch

padded_token_list, att_mask, token_type_ids, target = next(iter(train_dataloader))

print(padded_token_list.shape)
print(att_mask.shape)
print(token_type_ids.shape)
print(target.shape)

torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32])


In [34]:
# Get one val batch

padded_token_list, att_mask, token_type_ids, target = next(iter(val_dataloader))

print(padded_token_list.shape)
print(att_mask.shape)
print(token_type_ids.shape)
print(target.shape)

torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32])


In [35]:
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = len(list(label_to_ix.values())), 
    output_attentions = False,
    output_hidden_states = False)

# Send the model to the device.
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [36]:
# Get one train batch

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=8,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

batch = next(iter(train_dataloader))

b_input_ids = batch[0].to(device)
b_input_mask = batch[1].to(device)
b_token_type_ids = batch[2].to(device)
b_labels = batch[3].to(device)

In [37]:
outputs = model(b_input_ids, 
                token_type_ids=b_token_type_ids, 
                attention_mask=b_input_mask,
                labels=b_labels)

In [38]:
outputs

SequenceClassifierOutput([('loss',
                           tensor(2.6252, device='xla:1', grad_fn=<NllLossBackward>)),
                          ('logits',
                           tensor([[ 0.2286,  0.1942,  0.0867,  0.0607, -0.1505,  0.0328,  0.1157, -0.0560,
                                     0.3412, -0.2182,  0.0070, -0.1490, -0.1705],
                                   [ 0.1526,  0.1709,  0.0410,  0.0819, -0.1269,  0.0329,  0.0905, -0.0598,
                                     0.2985, -0.2244, -0.0078, -0.0790, -0.1738],
                                   [ 0.2011,  0.1890,  0.0678,  0.0752, -0.1296,  0.0313,  0.1105, -0.0450,
                                     0.2990, -0.2210, -0.0145, -0.1141, -0.1702],
                                   [ 0.1865,  0.2054,  0.0766,  0.0770, -0.1439, -0.0028,  0.1385, -0.0488,
                                     0.3504, -0.2388, -0.0451, -0.1007, -0.1458],
                                   [ 0.1860,  0.1800,  0.0509,  0.0583, -0.1236, 

In [39]:
len(outputs)

2

In [40]:
preds = outputs[1].detach().cpu().numpy()

y_true = b_labels.detach().cpu().numpy()
y_pred = np.argmax(preds, axis=1)

y_pred

array([8, 8, 8, 8, 8, 8, 8, 8])

In [41]:
# This is the accuracy without any fine tuning.

val_acc = accuracy_score(y_true, y_pred)

val_acc

0.0

In [42]:
%%time


# Set a seed value.
seed_val = 1024

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



# Store the accuracy scores for each fold model in this list.
# [[model_0 scores], [model_1 scores], [model_2 scores], [model_3 scores], [model_4 scores]]
# [[ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...], [ecpoch 1, epoch 2, ...]]

# Create a list of lists to store the val acc results.
# The number of items in this list will correspond to
# the number of folds that the model is being trained on.
fold_val_acc_list = []
for i in range(0, NUM_FOLDS):
    
    # append an empty list
    fold_val_acc_list.append([])
    
    
    
    

# For each epoch...
for epoch in range(0, NUM_EPOCHS):
    
    print("\nNum folds used for training:", NUM_FOLDS_TO_TRAIN)
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    # Get the number of folds
    num_folds = len(train_df_list)

    # For this epoch, store the val acc scores for each fold in this list.
    # We will use this list to calculate the cv at the end of the epoch.
    epoch_acc_scores_list = []
    
    # For each fold...
    for fold_index in range(0, NUM_FOLDS_TO_TRAIN):
        
        print('\n== Fold Model', fold_index)
        
        
        # .........................
        # Load the fold model
        # .........................
        
        if epoch == 0:
            
            # define the model
            model = BertForSequenceClassification.from_pretrained(
            MODEL_TYPE, 
            num_labels = len(list(label_to_ix.values())),       
            output_attentions = False, 
            output_hidden_states = False,
            )
            
            # Send the model to the device.
            model.to(device)
            
            optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8
            )
            
        else:
        
            # Get the fold model
            path_model = 'model_' + str(fold_index) + '.bin'
            model.load_state_dict(torch.load(path_model))

            # Send the model to the device.
            model.to(device)
        
        
        
        # .....................................
        # Set up the train and val dataloaders
        # .....................................
        
        
        # Intialize the fold dataframes
        df_train = train_df_list[fold_index]
        df_val = val_df_list[fold_index]
        
        # Reset the indices or the dataloader won't work.
        df_train = df_train.reset_index(drop=True)
        df_val = df_val.reset_index(drop=True)
    
        # Create the dataloaders
        train_data = BookDataset(df_train)
        val_data = BookDataset(df_val)

        train_dataloader = torch.utils.data.DataLoader(train_data,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                               num_workers=NUM_CORES)

        val_dataloader = torch.utils.data.DataLoader(val_data,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                               num_workers=NUM_CORES)
    
    
    

       

        # ========================================
        #               Training
        # ========================================
        
        stacked_val_labels = []
        targets_list = []

        print('Training...')

        # put the model into train mode
        model.train()

        # This turns gradient calculations on and off.
        torch.set_grad_enabled(True)


        # Reset the total loss for this epoch.
        total_train_loss = 0

        for i, batch in enumerate(train_dataloader):

            train_status = 'Batch ' + str(i+1) + ' of ' + str(len(train_dataloader))

            print(train_status, end='\r')


            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_token_type_ids = batch[2].to(device)
            b_labels = batch[3].to(device)

            model.zero_grad()        


            outputs = model(b_input_ids, 
                        token_type_ids=b_token_type_ids, 
                        attention_mask=b_input_mask,
                        labels=b_labels)

            # Get the loss from the outputs tuple: (loss, logits)
            loss = outputs[0]

            # Convert the loss from a torch tensor to a number.
            # Calculate the total loss.
            total_train_loss = total_train_loss + loss.item()

            # Zero the gradients
            optimizer.zero_grad()

            # Perform a backward pass to calculate the gradients.
            loss.backward()
            
            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Use the optimizer to update Weights
            
            # Optimizer for GPU
            # optimizer.step() 
            
            # Optimizer for TPU
            # https://pytorch.org/xla/
            xm.optimizer_step(optimizer, barrier=True)
            
           


        print('Train loss:' ,total_train_loss)


        # ========================================
        #               Validation
        # ========================================

        print('\nValidation...')

        # Put the model in evaluation mode.
        model.eval()

        # Turn off the gradient calculations.
        # This tells the model not to compute or store gradients.
        # This step saves memory and speeds up validation.
        torch.set_grad_enabled(False)


        # Reset the total loss for this epoch.
        total_val_loss = 0


        for j, val_batch in enumerate(val_dataloader):

            val_status = 'Batch ' + str(j+1) + ' of ' + str(len(val_dataloader))

            print(val_status, end='\r')

            b_input_ids = val_batch[0].to(device)
            b_input_mask = val_batch[1].to(device)
            b_token_type_ids = val_batch[2].to(device)
            b_labels = val_batch[3].to(device)      


            outputs = model(b_input_ids, 
                    token_type_ids=b_token_type_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)

            # Get the loss from the outputs tuple: (loss, logits)
            loss = outputs[0]

            # Convert the loss from a torch tensor to a number.
            # Calculate the total loss.
            total_val_loss = total_val_loss + loss.item()

            # Get the preds
            preds = outputs[1]


            # Move preds to the CPU
            val_preds = preds.detach().cpu().numpy()

            # Move the labels to the cpu
            targets_np = b_labels.to('cpu').numpy()

            # Append the labels to a numpy list
            targets_list.extend(targets_np)

            if j == 0:  # first batch
                stacked_val_preds = val_preds

            else:
                stacked_val_preds = np.vstack((stacked_val_preds, val_preds))
                
                
                
        # .........................................
        # Calculate the val accuracy for this fold
        # .........................................      


        # Calculate the validation accuracy
        y_true = targets_list
        y_pred = np.argmax(stacked_val_preds, axis=1)

        val_acc = accuracy_score(y_true, y_pred)
        
        
        epoch_acc_scores_list.append(val_acc)


        print('Val loss:' ,total_val_loss)
        print('Val acc: ', val_acc)
        
        
        # .........................
        # Save the best model
        # .........................
        
        if epoch == 0:
            
            # Save the Model
            model_name = 'model_' + str(fold_index) + '.bin'
            torch.save(model.state_dict(), model_name)
            print('Saved model as ', model_name)
            
        if epoch != 0:
        
            val_acc_list = fold_val_acc_list[fold_index]
            best_val_acc = max(val_acc_list)
            
            if val_acc > best_val_acc:
                # save the model
                model_name = 'model_' + str(fold_index) + '.bin'
                torch.save(model.state_dict(), model_name)
                print('Val acc improved. Saved model as ', model_name)
                
                
                
        # .....................................
        # Save the val_acc for this fold model
        # .....................................
        
        # Note: Don't do this before the above 'Save Model' code or 
        # the save model code won't work. This is because the best_val_acc will
        # become current val accuracy.
                
        # fold_val_acc_list is a list of lists.
        # Each fold model has it's own list corresponding to the fold index.
        # Here we choose a list corresponding to the fold number and append the acc score to that list.
        fold_val_acc_list[fold_index].append(val_acc)
        
            

        # Use the garbage collector to save memory.
        gc.collect()
        
        
    # .............................................................
    # Calculate the CV accuracy score over all folds in this epoch
    # .............................................................   
        
        
    # Print the average val accuracy for all 5 folds
    cv_acc = sum(epoch_acc_scores_list)/NUM_FOLDS_TO_TRAIN
    print("\nCV Acc:", cv_acc)


Num folds used for training: 3
======== Epoch 1 / 3 ========

== Fold Model 0


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training...
Train loss: 893.1927293241024

Validation...
Val loss: 79.39912250638008
Val acc:  0.9002998500749625
Saved model as  model_0.bin

== Fold Model 1


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training...
Train loss: 809.5130301490426

Validation...
Val loss: 77.70626430213451
Val acc:  0.8970014992503749
Saved model as  model_1.bin

== Fold Model 2


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training...
Train loss: 848.8112604618073

Validation...
Val loss: 71.49253100156784
Val acc:  0.9056971514242879
Saved model as  model_2.bin

CV Acc: 0.9009995002498751

Num folds used for training: 3
======== Epoch 2 / 3 ========

== Fold Model 0
Training...
Train loss: 208.2853031437844

Validation...
Val loss: 48.37440137937665
Val acc:  0.9307346326836582
Val acc improved. Saved model as  model_0.bin

== Fold Model 1
Training...
Train loss: 205.85150121338665

Validation...
Val loss: 38.16099697351456
Val acc:  0.948575712143928
Val acc improved. Saved model as  model_1.bin

== Fold Model 2
Training...
Train loss: 209.47454068809748

Validation...
Val loss: 35.10071453638375
Val acc:  0.950824587706147
Val acc improved. Saved model as  model_2.bin

CV Acc: 0.9433783108445777

Num folds used for training: 3
======== Epoch 3 / 3 ========

== Fold Model 0
Training...
Train loss: 103.5512673589401

Validation...
Val loss: 36.995951109565794
Val acc:  0.9476761619190405
Val acc improve

In [43]:
!ls

drive
model_0.bin
model_1.bin
model_2.bin
pytorch-xla-env-setup.py
sample_data
torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torchvision-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl


In [44]:
# Display the accuracy scores for each fold model.
# For info: 
# Fold model 0 is only training on fold 0 in each epoch.
# The same applies to the other fold models.

fold_val_acc_list

[[0.9002998500749625, 0.9307346326836582, 0.9476761619190405],
 [0.8970014992503749, 0.948575712143928, 0.9497751124437781],
 [0.9056971514242879, 0.950824587706147, 0.9488755622188906],
 [],
 []]

In [48]:
MODEL_TYPE = 'xlm-roberta-base'


L_RATE = 1e-5
MAX_LEN = 256 #256

NUM_EPOCHS = 3
BATCH_SIZE = 32 #32
NUM_CORES = os.cpu_count()

NUM_CORES

2

In [49]:
device = xm.xla_device()

print(device)

xla:1


In [50]:
df_train = train_df_list[0]

df_train.head()

,text,label
21551,work and fondling her poodle when all the fair...,arthur_doyle
29546,mind the only time that i ever fancy myself un...,jane_austen
1676,the cliff ’ ‘ what be that level line of littl...,thomas_hardy
10471,large apartment roof floor open on three side ...,mark_twain
23142,whereon wa write a notice that the occupier se...,arthur_doyle


In [51]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

# xlm-roberta-large
print('Loading XLMRoberta tokenizer...')
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)


Loading XLMRoberta tokenizer...


In [52]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [53]:
class BookDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'text']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    sentence1,           # Sentences to encode.
                    add_special_tokens = True,      # Add the special tokens.
                    max_length = MAX_LEN,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )
        
        # These are torch tensors.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # Convert the target to a torch tensor
        target = torch.tensor(label_to_ix[self.df_data.loc[index, 'label']])

        sample = (padded_token_list, att_mask, target)


        return sample


    def __len__(self):
        return len(self.df_data)

In [54]:
train_data = BookDataset(df_train)
val_data = BookDataset(df_val)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

val_dataloader = torch.utils.data.DataLoader(val_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)




print(len(train_dataloader))
print(len(val_dataloader))


834
209


In [55]:
from transformers import XLMRobertaForSequenceClassification

model = XLMRobertaForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = len(list(label_to_ix.values())), # The number of output labels. 2 for binary classification.
)

# Send the model to the device.
model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [56]:
# Create a batch of train samples
# We will set a small batch size of 8 so that the model's output can be easily displayed.

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=8,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

b_input_ids, b_input_mask, b_labels = next(iter(train_dataloader))

print(b_input_ids.shape)
print(b_input_mask.shape)
print(b_labels.shape)

torch.Size([8, 256])
torch.Size([8, 256])
torch.Size([8])


In [57]:
# Pass a batch of train samples to the model.

batch = next(iter(train_dataloader))

# Send the data to the device
b_input_ids = batch[0].to(device)
b_input_mask = batch[1].to(device)
b_labels = batch[2].to(device)

# Run the model
outputs = model(b_input_ids, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

# The ouput is a tuple (loss, preds).
outputs

SequenceClassifierOutput([('loss', tensor(2.4561, device='xla:1')),
                          ('logits',
                           tensor([[-0.4175,  0.0644, -0.1566, -0.0722, -0.1279,  0.1817, -0.3389,  0.0524,
                                    -0.1518, -0.1375,  0.0508,  0.1477, -0.2201],
                                   [-0.4152,  0.0684, -0.1553, -0.0703, -0.1225,  0.1816, -0.3471,  0.0504,
                                    -0.1494, -0.1424,  0.0461,  0.1557, -0.2161],
                                   [-0.4128,  0.0733, -0.1573, -0.0694, -0.1246,  0.1785, -0.3462,  0.0543,
                                    -0.1490, -0.1387,  0.0519,  0.1544, -0.2184],
                                   [-0.4182,  0.0727, -0.1596, -0.0681, -0.1222,  0.1790, -0.3419,  0.0525,
                                    -0.1483, -0.1367,  0.0499,  0.1492, -0.2180],
                                   [-0.4185,  0.0690, -0.1567, -0.0685, -0.1252,  0.1812, -0.3436,  0.0533,
                           

In [58]:
preds = outputs[1].detach().cpu().numpy()

y_true = b_labels.detach().cpu().numpy()
y_pred = np.argmax(preds, axis=1)

y_pred

array([5, 5, 5, 5, 5, 5, 5, 5])

In [59]:
# This is the accuracy without fine tuning.

val_acc = accuracy_score(y_true, y_pred)

val_acc

0.25

In [60]:
# Define the optimizer
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

In [61]:
# Create the dataloaders.

train_data = BookDataset(df_train)
val_data = BookDataset(df_val)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

val_dataloader = torch.utils.data.DataLoader(val_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)




print(len(train_dataloader))
print(len(val_dataloader))


834
209


In [ ]:
%%time


# Set the seed.
seed_val = 101

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []


# For each epoch...
for epoch in range(0, NUM_EPOCHS):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    

    stacked_val_labels = []
    targets_list = []

    # ========================================
    #               Training
    # ========================================
    
    print('Training...')
    
    # put the model into train mode
    model.train()
    
    # This turns gradient calculations on and off.
    torch.set_grad_enabled(True)


    # Reset the total loss for this epoch.
    total_train_loss = 0

    for i, batch in enumerate(train_dataloader):
        
        train_status = 'Batch ' + str(i) + ' of ' + str(len(train_dataloader))
        
        print(train_status, end='\r')


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        


        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
        
        # Get the loss from the outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # Convert the loss from a torch tensor to a number.
        # Calculate the total loss.
        total_train_loss = total_train_loss + loss.item()
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        
        
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        
        
        # Use the optimizer to update the weights.
        
        # Optimizer for GPU
        # optimizer.step() 
        
        # Optimizer for TPU
        # https://pytorch.org/xla/
        xm.optimizer_step(optimizer, barrier=True)

    
    print('Train loss:' ,total_train_loss)


    # ========================================
    #               Validation
    # ========================================
    
    print('\nValidation...')

    # Put the model in evaluation mode.
    model.eval()

    # Turn off the gradient calculations.
    # This tells the model not to compute or store gradients.
    # This step saves memory and speeds up validation.
    torch.set_grad_enabled(False)
    
    
    # Reset the total loss for this epoch.
    total_val_loss = 0
    

    for j, batch in enumerate(val_dataloader):
        
        val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
        
        print(val_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)
        
        # Get the loss from the outputs tuple: (loss, logits)
        loss = outputs[0]
        
        # Convert the loss from a torch tensor to a number.
        # Calculate the total loss.
        total_val_loss = total_val_loss + loss.item()
        

        # Get the preds
        preds = outputs[1]


        # Move preds to the CPU
        val_preds = preds.detach().cpu().numpy()
        
        # Move the labels to the cpu
        targets_np = b_labels.to('cpu').numpy()

        # Append the labels to a numpy list
        targets_list.extend(targets_np)

        if j == 0:  # first batch
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

    
    # Calculate the validation accuracy
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    
    val_acc = accuracy_score(y_true, y_pred)
    
    
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)


    # Save the Model
    torch.save(model.state_dict(), 'model.pt')
    
    # Use the garbage collector to save memory.
    gc.collect()


======== Epoch 1 / 3 ========
Training...
